In [17]:
import requests
import folium
import polyline
import json


In [18]:

def get_route(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat):    
    loc = "{},{};{},{}".format(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
    url = "http://router.project-osrm.org/route/v1/driving/"
    r = requests.get(url + loc) 
    if r.status_code!= 200:
        return {}
    res = r.json()   
    routes = polyline.decode(res['routes'][0]['geometry'])
    start_point = [res['waypoints'][0]['location'][1], res['waypoints'][0]['location'][0]]
    end_point = [res['waypoints'][1]['location'][1], res['waypoints'][1]['location'][0]]
    distance = res['routes'][0]['distance']
    
    out = {'route':routes,
           'start_point':start_point,
           'end_point':end_point,
           'distance':distance
          }

    return out


In [19]:

    
def get_map(route, line=True):
    
    m = folium.Map(location=[(route['start_point'][0] + route['end_point'][0])/2, 
                             (route['start_point'][1] + route['end_point'][1])/2], 
                   zoom_start=13)

    if line:
        folium.PolyLine(
            route['route'],
            weight=8,
            color='blue',
            opacity=0.6
        ).add_to(m)
    else:
        for idx, marker in enumerate(route['route']):
            folium.Marker(
            location=(marker[1], marker[0]),
            icon=folium.Icon(icon='play', color=route['route_colors'][idx])
        ).add_to(m)

    folium.Marker(
        location=route['start_point'],
        icon=folium.Icon(icon='play', color='green')
    ).add_to(m)

    folium.Marker(
        location=route['end_point'],
        icon=folium.Icon(icon='stop', color='red')
    ).add_to(m)

    return m


In [20]:

pickup_lon, pickup_lat, dropoff_lon, dropoff_lat = 19.8512,45.246,19.841270,45.253672

with open('route_visited.json', 'r') as j:
     r = json.loads(j.read())['arr']
     rt = r[0]
     rc = r[1]
test_route = {
    'route': rt,
    'start_point': (rt[0][1], rt[0][0]),
    'end_point': ((rt[-1][1], rt[-1][0])),
    'route_colors': rc
}
m=get_map(test_route, line=False)

In [21]:
test_route

{'route': [[19.83011, 45.25826],
  [19.83011, 45.25826],
  [19.83042, 45.25828],
  [19.83042, 45.25828],
  [19.8306, 45.25806],
  [19.8306, 45.25806],
  [19.8308, 45.2581],
  [19.8308, 45.2581],
  [19.83059, 45.25849],
  [19.83059, 45.25849],
  [19.830978333333334, 45.25859500000001],
  [19.831366666666668, 45.258700000000005],
  [19.831755, 45.258805],
  [19.832143333333335, 45.25891000000001],
  [19.832531666666668, 45.259015000000005],
  [19.83292, 45.25912],
  [19.83292, 45.25912],
  [19.833105652173916, 45.258786739130436],
  [19.833291304347828, 45.25845347826087],
  [19.83347695652174, 45.2581202173913],
  [19.83366260869565, 45.257786956521734],
  [19.833848260869566, 45.25745369565217],
  [19.834033913043477, 45.2571204347826],
  [19.83421956521739, 45.25678717391303],
  [19.8344052173913, 45.256453913043465],
  [19.834590869565215, 45.2561206521739],
  [19.834776521739126, 45.25578739130433],
  [19.83496217391304, 45.255454130434764],
  [19.835147826086953, 45.2551208695652],

In [22]:
m